# Challenge 05 - Red Teaming

## 1. Overview 

## 2. Setting Up Environment 

In [7]:
# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI
import asyncio
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

## 2. Creating Red Teaming Agent

In [8]:
## Using Azure AI Foundry project from environment variable
azure_ai_project = os.getenv("AZURE_AI_PROJECT")

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential() # required
)

## 3. OpenAI Client and callback

In [9]:
# Create Azure OpenAI client using managed identity (recommended approach)
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI authentication"""
    credential = DefaultAzureCredential()
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

azure_openai_client = AzureOpenAI(
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_ad_token_provider=get_azure_ad_token
)

# Target callback function that uses your Azure OpenAI deployment
def azure_openai_callback(query: str) -> str:
    """
    This function represents your AI system that will be red-teamed.
    It sends the query to your Azure OpenAI deployment and returns the response.
    """
    try:
        response = azure_openai_client.chat.completions.create(
            model=os.getenv("CHAT_MODEL_NAME2"),  # This should match your deployment name
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": query}
            ],
            max_tokens=500,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

## 4. Running the Red Team Agent

In [ ]:
async def main():
    # The scan method expects a target function, not a configuration object
    red_team_result = await red_team_agent.scan(
        target=azure_openai_callback,
        risk_categories=[ # optional, defaults to all four risk categories
            RiskCategory.Violence,
            RiskCategory.HateUnfairness,
            RiskCategory.Sexual,
            RiskCategory.SelfHarm
        ],
        num_objectives=5  # Number of conversation turns per attack
    )
    print("Red team scan completed!")
    print(red_team_result)

if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        # If there's a running event loop (e.g., in Jupyter), use create_task
        task = loop.create_task(main())
    else:
        # Otherwise, safe to use asyncio.run
        asyncio.run(main())

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<main() done, defined at /tmp/ipykernel_4902/2305545484.py:1> exception=ClientAuthenticationError("DefaultAzureCredential failed to retrieve a token from the included credentials.\nAttempted credentials:\n\tEnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.\nVisit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.\n\tManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. invalid_request\n\tSharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.\n\tAzureCliCredential: Please run 'az login' to set up an account\n\tAzurePowerShellCredential: PowerShell is not installed\n\tAzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure

🚀 STARTING RED TEAM SCAN: None
📂 Output directory: ./.scan_20250801_185240
📊 Risk categories: ['hate_unfairness', 'sexual', 'violence', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/3bf1379d-e32e-42c7-afb8-b6062c1adc68?wsid=/subscriptions/27b8731d-0fd3-4f0a-9ebd-6fb73190cd8d/resourceGroups/red-teaming-test/providers/Microsoft.CognitiveServices/accounts/red-teaming-test-resource/projects/red-teaming-test&tid=16b3c013-d300-468d-ac64-7eda0820b6d3
📋 Planning 4 total tasks


Scanning:   0%|                  | 0/4 [00:00<?, ?scan/s, current=fetching baseline/hate_unfairness]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                           | 0/4 [00:00<?, ?scan/s, current=fetching baseline/sexual]

...

Strategy baseline, Risk hate_unfairness: Processed batch 3/4
Strategy baseline, Risk sexual: Processed batch 3/4
Strategy baseline, Risk violence: Processed batch 3/4
Strategy baseline, Risk self_harm: Processed batch 3/4


Scanning: 100%|██████████████████████████████████| 4/4 [04:49<00:00, 72.37s/scan, current=batch 1/1]


Evaluation results saved to "/workspace/066-OpenAIFundamentals/Student/Resources/notebooks/.scan_20250801_185240/baseline_hate_unfairness_7ad4b83b-f4fa-405b-b462-d34793c846f7.json".

✅ Completed task 1/4 (25.0%) - baseline/hate_unfairness in 287.8s
   Est. remaining: 14.6 minutes
Evaluation results saved to "/workspace/066-OpenAIFundamentals/Student/Resources/notebooks/.scan_20250801_185240/baseline_sexual_12f00f94-ee56-4d34-80b0-e999ff5873f0.json".

✅ Completed task 2/4 (50.0%) - baseline/sexual in 287.8s
   Est. remaining: 4.9 minutes
Evaluation results saved to "/workspace/066-OpenAIFundamentals/Student/Resources/notebooks/.scan_20250801_185240/baseline_violence_77703476-20cd-4006-bbf3-a6af82599102.json".

✅ Completed task 3/4 (75.0%) - baseline/violence in 287.8s
   Est. remaining: 1.6 minutes
Evaluation results saved to "/workspace/066-OpenAIFundamentals/Student/Resources/notebooks/.scan_20250801_185240/baseline_self_harm_9dcbf4f5-517b-4c51-bda7-7419d9cae9f8.json".

✅ Completed ta

Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
RequestId:873e4c37-d01e-0004-5f16-039fcc000000
Time:2025-08-01T18:57:39.2982315Z
ErrorCode:AuthorizationFailure
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthorizationFailure</Code><Message>This request is not authorized to perform this operation.
RequestId:873e4c37-d01e-0004-5f16-039fcc000000
Time:2025-08-01T18:57:39.2982315Z</Message></Error>


Evaluation results saved to "/workspace/066-OpenAIFundamentals/Student/Resources/notebooks/.scan_20250801_185240/final_results.json".

Overall ASR: 0.0%
Attack Success: 0/40 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Hate-unfairness   | 0.0%           | N/A                          | N/A                             | N/A                           
Sexual            | 0.0%           | N/A                          | N/A                             | N/A                           
Violence          | 0.0%           | N/A                          | N/A                             | N/A                           
Se